## Loading the dataset and basic exploration

In [ ]:
import numpy as np
from scipy import stats
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm

from statsmodels.graphics.api import qqplot

In [ ]:
df= pd.read_csv("sunspots.csv")
df.head(10)

In [ ]:
print(sm.datasets.sunspots.NOTE)

In [ ]:
df.shape

In [ ]:
df.info()

## Preparation of data

In [ ]:
#we will convert the year column into a date-time object
df.index = pd.Index(sm.tsa.datetools.dates_from_range('1700', '2008'))
del df["YEAR"]
df.head()

In [ ]:
#plotting the data o have a basic look at the distribution
# show plots in the notebook
%matplotlib inline
df.plot(figsize=(30,8));

In [ ]:
# Durbin-Watson statistic value lies in the 0-4 range, with a value near two indicating no first-order serial correlation. 
#Positive serial correlation is associated with DW values below 2 and negative serial correlation with DW values above 2.
sm.stats.durbin_watson(df)


The value of **Durbin-Watson** statistic in our example is 0.1395. That means that there is a strong evidence that the variable open has high autocorrelation.

In [ ]:
# plotting the ACF and PACF
%matplotlib inline
fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(df.values.squeeze(), lags=40, ax=ax1)
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(df, lags=40, ax=ax2)

 The plots indicate that *autocorrelation* is present

In [ ]:
#using pandas for ACF plots
from pandas.plotting import autocorrelation_plot
# show plots in the notebook
%matplotlib inline
df['SUNACTIVITY_2'] = df['SUNACTIVITY']
df['SUNACTIVITY_2'] = (df['SUNACTIVITY_2'] - df['SUNACTIVITY_2'].mean()) / (df['SUNACTIVITY_2'].std())
plt.acorr(df['SUNACTIVITY_2'],maxlags = len(df['SUNACTIVITY_2']) -1, linestyle = "solid", usevlines = False, marker='')
plt.show()
autocorrelation_plot(df['SUNACTIVITY'])
plt.show()

# Modeling the Data

from statsmodels.tsa.arima.model import ARIMA

arma_mod20 = ARIMA(df['SUNACTIVITY'], order=(1, 0, 0)).fit()
print(arma_mod20.params)


In [ ]:
from statsmodels.tsa.arima.model import ARIMA

arma_mod20 = ARIMA(df['SUNACTIVITY'], order=(1, 0, 0)).fit()
print(arma_mod20.params)


In [ ]:
print(arma_mod20.aic, arma_mod20.bic, arma_mod20.hqic)

In [ ]:
sm.stats.durbin_watson(arma_mod20.resid.values)

The Durbin-Watson test shows no autocorrelation.

## Plotting the Data

In [ ]:
# show plots in the notebook
%matplotlib inline
fig = plt.figure(figsize=(12,8))
ax = fig.add_subplot(111)
ax = arma_mod20.resid.plot(ax=ax);

## Analyzing the Residuals

In the following steps, we calculate the residuals, tests the null hypothesis that the residuals come from a normal distribution, and construct a qq-plot.

In [ ]:
resid20 = arma_mod20.resid
stats.normaltest(resid20)


In [ ]:
%matplotlib inline
fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(resid20.values.squeeze(), lags=40, ax=ax1)
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(resid20, lags=40, ax=ax2)

Next, we calculate the lag, autocorrelation (AC), Q statistic and Prob>Q. The Ljung–Box Q test (named for Greta M. Ljung and George E. P. Box) is a type of statistical test of whether any of a group of autocorrelations of a time series are different from zero. The null hypothesis is, H0: The data are independently distributed (i.e. the correlations in the population from which the sample is taken are 0, so that any observed correlations in the data result from randomness of the sampling process).

In [ ]:
r, q, p = sm.tsa.acf(resid20.values.squeeze(), nlags=24, qstat=True)
data = np.c_[range(1, 25), r[1:25], q, p]
table = pd.DataFrame(data, columns=['lag', "AC", "Q", "Prob(>Q)"])
print(table.set_index('lag'))


Notice that the p-values for the Ljung–Box Q test all are well above .05 for lags 1 through 8, indicating “significance.” This is not a desirable result. However, the p-values for the remaining lags through 40 data values as less than .05. So there is much data not contributing to correlations at high lags.


## Predictions

Next, we compute the predictions and analyze their fit against actual values.

In [ ]:
predict_sunspots20 = arma_mod20.predict('1990', '2012', dynamic=True)
print(predict_sunspots20)

In [ ]:
ax = df.loc['1950':].plot(figsize=(12,8))
ax = predict_sunspots20.plot(ax=ax, style='r--', label='Dynamic Prediction');
ax.legend();
ax.axis((-20.0, 38.0, -4.0, 200.0));

The fit looks good up to about 1998 and underfit the data afterwards.

## Calculate Forecast Errors

#### Mean absolute error:
The mean absolute error (MAE) value is computed as the average absolute error value. If this value is 0 (zero), the fit (forecast) is perfect. As compared to the mean squared error value, this measure of fit will “de-emphasize” outliers, that is, unique or rare large error values will affect the MAE less than the MSE value.

#### Mean Forecast Error (Bias).
The mean forecast error (MFE) is the average error in the observations. A large positive MFE means that the forecast is undershooting the actual observations, and a large negative MFE means the forecast is overshooting the actual observations. A value near zero is ideal.

The MAE is a better indicator of fit than the MFE.